In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
list_files = os.listdir('datasets/history-scottish-witchcraft/')

In [3]:
# Put all data files into a dictionary of dataframes for easy access
def create_dfs(list_files):
    dfs_dict = {}
    for filename in list_files:
        if filename[-4:] == '.csv':
            df_name = filename[4:-4]
            dfs_dict[df_name] = pd.read_csv('datasets/history-scottish-witchcraft/{}'.format(filename))
    return dfs_dict

In [4]:
all_dfs = create_dfs(list_files)

In [5]:
all_dfs.keys()

['MovestoHLA',
 'CalendarCustom',
 'Imprisonment',
 'MentionedAsWitch',
 'RitualObject',
 'Commission',
 'ReligiousMotif',
 'DevilAppearance',
 'Appeal',
 'OtherNamedwitch',
 'ShapeChanging',
 'Complaint',
 'WitchesMeetingPlace',
 'Torture',
 'Elf_FairyElements',
 'WeatherModification',
 'Ref_Parish',
 'Trial_Person',
 'Case',
 'PropertyDamage',
 'PrevCommission',
 'DemonicPact',
 'Ordeal',
 'Person',
 'Denunciation',
 'Trial',
 'WhiteMagic',
 'MusicalInstrument',
 'Reference',
 'Source',
 'Confession',
 'CounterStrategy',
 'Malice',
 'Case_person',
 'Accused',
 'LinkedTrial',
 'Accused_family',
 'OtherCharges']

In [6]:
accused = all_dfs['Accused']

In [7]:
accused.head(1)

,AccusedRef,AccusedSystemId,AccusedID,FirstName,LastName,M_Firstname,M_Surname,Alias,Patronymic,DesTitle,...,Res_NGR_Northing,Ethnic_origin,MaritalStatus,SocioecStatus,Occupation,Notes,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,A/EGD/10,EGD,10,Mareon,Quheitt,Marion,White,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SMD,05/15/01 11:06:51,jhm,08/09/02 11:40:51


In [10]:
family = all_dfs['Accused_family']

In [11]:
family.head(1)

,Accused_familyRef,Accused_familySystemId,Accused_familyID,Surname,Firstname,Alias,Patronymic,DesTitle,Est_Year_of_Birth,Age,Age_estcareer,Age_estchild,Occupation,Relationship,AccusedRef,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,AF/JO/1,JO,1,Leyis,John,NaN,NaN,NaN,NaN,NaN,0,0,stabler,husband,A/EGD/2067,jhm,05/16/01 14:45:56,jhm,05/16/01 14:48:31


In [12]:
family.shape

(951, 19)

In [13]:
mentioned = all_dfs['MentionedAsWitch']

In [14]:
mentioned.head(1)

,MentionRef,MentionSystemId,MentionID,Trialref,Mentiondate,Mentiondate_as_date,Mentiontype,TrialOfAccusedRef,TrialOfFirstName,TrialOfLastName,MentionedInTrialOfRef,dateOfMentionedTrial,dateOfMentionedTrial_as_date,FateOfMentionedWitch,Notes,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,MW/JO/1,JO,1,T/JO/4,23/2/1597,02/23/97 00:00:00,Accomplice,A/EGD/2068,Thomas,Leyis,T/JO/3,17/2/1597,02/17/97 00:00:00,Burnt,NaN,jhm,05/17/01 16:31:34,LEM,09/09/02 15:07:04


In [15]:
mentioned.Mentiontype.unique()

array(['Accomplice', 'Denounced', 'Witchcraft Precedent', 'Known Witch',
       nan, 'Previously Tried', 'Consulted', 'Hired'], dtype=object)

In [16]:
mentioned.FateOfMentionedWitch.unique()

array(['Burnt', nan, 'Banished', 'Set free', 'Strangled/burnt', 'executed',
       'Dead', 'Died (natural causes)', 'Executed', 'Died in Prison',
       'Fugitive', 'Suicide in prison', 'Died', 'Suicide'], dtype=object)

In [17]:
trial = all_dfs['Trial']

In [18]:
trial.head(1)

,Trialref,TrialId,TrialSystemId,CaseRef,TrialType,Trial_settlement,Trial_parish,Trial_presbytery,Trial_county,Trial_burgh,...,Exec_county,Exec_burgh,Exec_NGR_Letters,Exec_NGR_Easting,Exec_NGR_Northing,PostTrialNotes,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,T/JO/1,1,JO,C/EGD/2120,2,NaN,NaN,Aberdeen,Aberdeen,Aberdeen,...,NaN,Aberdeen,NaN,NaN,NaN,NaN,jhm,05/16/01 15:04:51,jhm,10/24/02 09:32:53


In [19]:
malice = all_dfs['Malice']

In [20]:
malice.head(1)

,MaliceRef,MaliceSystemId,MaliceID,CaseRef,Causeofmalice,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,MA/JO/1,JO,1,C/EGD/2120,Failed business interaction,jhm,05/16/01 17:01:25,jhm,05/16/01 17:05:02


In [21]:
malice.Causeofmalice.unique()

array(['Failed business interaction', 'Debt', 'Jealousy of Neighbours',
       'Revenge', 'Grudge', 'Slander', 'Social slight', 'Physical insult',
       'Exclusion from social events', 'Refusal of alms', nan,
       'Business interactions', 'Failed healing',
       'Revenge for being called a witch', 'Verbal Insult', 'Taunting',
       'Envy'], dtype=object)

In [22]:
confession = all_dfs['Confession']

In [23]:
confession.head(1)

,ConfessionRef,ConfessionSystemId,ConfessionID,Trialref,CentralTrialConfession,Confessionrec,Confessiondate,Confessiondate_as_date,Confessionplace,Confessionlocation,Confessiontext,Confessiondetails,Confessionret,Notes,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,CF/JO/1,JO,1,T/JO/2,0,1,19/4/1643,04/19/43 00:00:00,Church,Haddington,0,1,0,NaN,jhm,05/17/01 12:11:47,jhm,05/17/01 12:12:27


In [24]:
moves = all_dfs['MovestoHLA']

In [25]:
moves.head(1)

,MoveRef,MoveSystemId,MoveID,Trialref,Moveto,Moveexists,Movedate,Movedate_as_date,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,HLA/JO/10,JO,10,T/JO/342,Bishop,1,17/5/1632,05/17/32 00:00:00,jhm,07/04/01 13:35:08,jhm,07/04/01 13:35:18


In [26]:
torture = all_dfs['Torture']

In [27]:
torture.Torturetype.unique()

array(['Burning feet', 'Various', 'Sleep Deprivation', 'Bound with ropes',
       'Whip', 'Hanging by thumbs', nan, 'Stocks', 'Irons', 'Bow strings',
       'Wedges on the shins', 'Cashielaws', 'Haircloth', 'Tied to pole'], dtype=object)

In [28]:
imprisonment = all_dfs["Imprisonment"]

In [29]:
imprisonment.head(1)

,ImprisonRef,ImprisonSystemId,ImprisonID,Trialref,CentralTrialImprison,Imprisondate,Imprisondate_as_date,Prison,Location,Moving,Fate_in_prison,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,IM/JO/10,JO,10,T/JO/78,0,NaN,NaN,Tolbooth,Haddington,0,NaN,jhm,06/07/01 16:20:29,LEM,10/28/02 12:45:55


In [30]:
devil_appearance = all_dfs['DevilAppearance']

In [31]:
devil_by_type = devil_appearance.groupby("Devil_Type")['CaseRef'].count()

In [32]:
devil_by_type.sort_values(ascending=False)

Devil_Type
Male                      250
Animal Devil               60
Female                     26
Spirit                     17
Unspecified Devil           9
Male Fairy                  9
Female Fairy                6
Ghost                       5
Fairy                       4
Inanimate Object Devil      2
Human Devil                 2
Other Demon                 1
Insect Devil                1
Child Devil                 1
Baby                        1
Name: CaseRef, dtype: int64

In [34]:
case = all_dfs['Case']

In [40]:
print case.columns.tolist()

['CaseRef', 'CaseId', 'CaseSystemId', 'NamedIndividual', 'AccusedRef', 'CaseStart_date', 'CaseStart_date_as_date', 'Case_date', 'Case_date_as_date', 'Age_at_case', 'CaseCommonName', 'Commission', 'Complaint', 'Correspondence', 'Chronicle', 'Other', 'Suspects_text', 'UNorthodoxRelPract_p', 'UNorthodoxRelPract_s', 'Consulting_p', 'Consulting_s', 'Demonic_p', 'Demonic_s', 'Demonic_possess_p', 'Demonic_possess_s', 'Fairies_p', 'Fairies_s', 'Folk_healing_p', 'Folk_healing_s', 'Maleficium_p', 'Maleficium_s', 'Midwifery_p', 'Midwifery_s', 'ImplicatedByAnother_p', 'ImplicatedByAnother_s', 'Neighbhd_dispute_p', 'Neighbhd_dispute_s', 'PoliticalMotive_p', 'PoliticalMotive_s', 'PropertyMotive_p', 'PropertyMotive_s', 'RefusedCharity_p', 'RefusedCharity_s', 'Treason_p', 'Treason_s', 'Other_p', 'Other_s', 'OtherText', 'NotEnoughInfo_p', 'NotEnoughInfo_s', 'WhiteMagic_p', 'WhiteMagic_s', 'Charnotes', 'DemonicPact', 'DevilNotes', 'WitchesMeeting', 'MeetingName', 'DevilPresent', 'Maleficium', 'CommunalS

In [51]:
case["OtherChargesNotes"].unique()

array([nan,
       'The details in the presbytery records are all about charming. Witchcraft is not mentioned.',
       'Incest with his mother and sister for two years',
       'charged with charming in RPC p. 104',
       'In trouble in 1617 charged with adultery.',
       "Also accused of having sex (and a baby) with her sister's husband.",
       'She confessed to both',
       'Presbytery accepted that he was guilty of charming and witchcraft in 1643 and ordered him to make public satisfaction.',
       'Named as witch and charmer by Janet Coutts',
       'Presbytery minutes note that she had a written book of her pretended prophecies.',
       'no details',
       'Accused of murdering two women by poisoning and also of murdering her own (first?) husband.',
       'Entry in presbytery notes that accused had used charming many times previously.',
       'Main accusation according to presbytery was blasphemy and devilish delusions which later became devilry, blasphemy and witchcraf

In [53]:
case['OtherMaleficiaNotes'].unique()

array([nan,
       'Said to have been part of the plot to sink the ships of James VI and his wife Anne of Denmark. Claimed by others that they raised storms and winds.',
       'Claimed by other confessing witches that she burned down the Mill; destroyed ships during a herring drove',
       'Claimed by another confessing witch that she burned down a mill; destroyed ships during a herring drove',
       "Part of the witches' crew that tried to kill Ann of Denmark.",
       'Allegedly sunk a ship.',
       'It was claimed by others that she was involved in trying to sink the royal ships.',
       "Supposedly raised a storm in Newhaven harbour.  She predicted that there wouldn't be enough salt for the Dunbar herring drove.",
       "She was accused of being an accomplice to Gourley's (c/egd/308) firestarting.  She claimed that she and Gourley took 5 burning coals and put them in places of the house that burnt down.",
       'burned a mill; destroyed ships during a herring drove',
       

In [54]:
case['CaseNotes']

0                          certain persons from Haymouth.
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
5                                                     NaN
6       A very complicated case that moved back and fo...
7                                                     NaN
8                                                     NaN
9                                                     NaN
10                                                    NaN
11                                                    NaN
12                                                    NaN
13                                                    NaN
14                                                    NaN
15                                                    NaN
16                                                    NaN
17            

In [56]:
case['OtherText'].unique()

array([nan, 'Sorcery', 'witch for hire', 'Murder', 'enchanted drink',
       'Weird conspiracy to get a woman', 'Incest',
       'Weird conspiracy to get a woman.', 'Theft',
       "turned state's evidence for prosecution", 'Not known', 'Blasphemy',
       'incest', 'murder', 'Slander', 'slander', 'sexual misconduct',
       'theft', 'witchcraft and rape', 'Attempted Suicide/Murder'], dtype=object)

In [60]:
case['DefenseNotes'].unique()

array([nan,
       'Appealed to Privy Council to change the location of their trial to Edinburgh, rather than Haddington. Questioned the competency of the local commissioners. Appeal denied.',
       "A doctor testified that she had symptoms of 'hyprocondriack distraction'",
       'No words or deeds; admits to blasphemy',
       'She used the common defence strategy of trying to discredit the witnesses.',
       'Claimed that the only thing in her dittay that was an offence of any kind was the ritual with the nettle and the urine which the defence argued was a superstitious rite not witchcraft.  Lots of detailed legal pleadings, very rich on this.',
       'Argued that no words or deeds were alleged; use of ridicule to challenge witnesses',
       'Accused petitioned the PC on the grounds that she was too old to be in prison and that there were no grounds to the accusation other than the word of an executed person. She complained that she had had a good name and had no bad reputation.

In [66]:
case.sum()

CaseId                         6236097.0
NamedIndividual                  -3212.0
Age_at_case                       7302.0
Chronicle                            0.0
Suspects_text                     1541.0
UNorthodoxRelPract_p                 0.0
UNorthodoxRelPract_s                85.0
Consulting_p                        11.0
Consulting_s                        46.0
Demonic_p                           49.0
Demonic_s                          528.0
Demonic_possess_p                   56.0
Demonic_possess_s                   79.0
Fairies_p                            9.0
Fairies_s                          113.0
Folk_healing_p                      40.0
Folk_healing_s                     141.0
Maleficium_p                        40.0
Maleficium_s                       414.0
Midwifery_p                          0.0
Midwifery_s                         10.0
ImplicatedByAnother_p               32.0
ImplicatedByAnother_s              948.0
Neighbhd_dispute_p                  49.0
Neighbhd_dispute

In [68]:
case['Chronicle'].unique()

array([ nan])

In [70]:
case['Commission'].unique()

array(['commission for execution', nan,
       'commission for trial and execution', "Gardener's wife",
       'to try witches in Queensferry', 'Privy Council',
       'justiciary comm. Issued by parliament',
       'justiciary comm issued by parliament',
       'commission for execution issued by parliament',
       'commission for execution from parliament.', 'Issued by parliament',
       'Privy Counil', 'Sum wemen were burned', 'certane wemen',
       'Sundry persons', 'Divers persons', 'Dyvers both men and weomen',
       'open commission to try witches',
       'for the general assembly to fill in commissioners',
       'all requests for commissions to go throughthe Privy Council',
       'granting of full power of commissions to a commissioner',
       'revocation of commissions for witchcraft',
       'of Lieutenancy in the West March to Lord Ochiltree',
       'of justiciary and lieutenancy', 'Sundry witches', 'divers persons',
       'Procedural change requiring Bishops to re

In [72]:
case['Complaint'].unique()

array([nan, 'against a witch-pricker',
       'bailies complaint about friends of witches visiting them in ward',
       'About a common ritual'], dtype=object)

In [74]:
case['Correspondence'].unique()

array([nan, "'some others' to be executed", '3 witches out of Caithness',
       'many witches are taken and burnt in the Merse',
       'Dutch gentlewoman from Lubeck',
       'King occupied in examining witches', 'about examining witches',
       'Some of the worst sort of witches escaped to England',
       'Woman, and many others',
       'description of scepticism about 1590-1 withcraft trials',
       'commentary about 1590-1',
       'instructions from James VI concerning witches',
       'about James VI and witches', 'reportage about Ewphame Mackalyean',
       'Sondry witches taken', 'Reportage about the 1597 trials',
       'King James is interested in a great number of witches',
       'From King to the Privy Council', 'King to the Advocate',
       'King to Lord of Lorne',
       'Description of Torture written to the Speaker of the Parliament in Westminster'], dtype=object)

In [76]:
case['Other'].unique()

array([nan, 'Kirk Session minutes', 'Burgh Council Minutes',
       'Presbytery entry', 'kirk session', 'Presbytery records',
       'Presbytery minutes', 'kirk session records',
       'specification of witchcraft under jurisdiction of circuit courts',
       'Arrest of C. Cauldwell (alias John Dickson) as a witchpricker',
       'Court Book of Orkney', 'Court book of Shetland',
       'Statute forbidding the concealment of witchcraft',
       'Shetland law forbidding the concealment of witchcraft',
       'John Kincaid, witch pricker examined', 'General Assembly policy',
       'Ayre Burgh Accounts', 'Some persons in Dreghorn',
       "Minister's announcement to pursue witches",
       'Church of Scotland petition to Parliament',
       'Presbytery Querry re: witch finders',
       'for tryal and torture of witches',
       'Bailie requests ministers to wait upon 12 condemed witches',
       'Presbytery request for commission',
       'Bailie complaint to Presbytery about their impro

In [79]:
case['Charnotes'].unique()

array([nan,
       'Part of the North Berwick witch hunt.  It involved many people and many accustions about witches meetings, specifically two separate attempts to sink the royal ships.  The first to sink Queen Anne on her way home, second to trouble King James after he went to get her in Denmark.',
       'Part of the North Berwick witch hunt.  It involved many people and many accustions about witches meetings, specifically two separate attempts to sink the royal ships.  The first to sink Queen Anne on her way home, the second to trouble King James after he went to get her in Denmark.',
       'The main accusations appear to relate to healing rituals and advice that did not work and the patients died rather than death as a result of maleficium. No demonic features.',
       'Likely to be demonic but not enough information.',
       'Demonic pact and charming confessed so included but few details.',
       'Pact and charming confessed but few details.',
       'Demonic pact and charmi

In [81]:
case['DevilNotes'].unique()

array([nan, "Said by others to have met the devil as a 'mekle black man'",
       'Conversing with the Devil',
       "Others confessed that she met the devil as a 'mekle black man'.",
       'Baptised a cat.', 'No details of pact.', 'No pact details',
       'no details about pact recorded.',
       'paction confessed but no details.',
       'Pact confessed but no details.',
       'Presbytery notes confessed paction with the devil.',
       'Few details but presbytery notes she confessed paction with the devil.',
       'Her servant saw a vision of a man in a white shirt with a tail.',
       'Confessed that he had seen the fairies in different forms but did not describe them.',
       'Dalkeith presbytery note the accused confessed to paction, witchcraft and charming.',
       'Devil gave her the power to heal.',
       "Denounced Bothwell.  She said the devil would kill the king for Lord Bothwell's sake.",
       "Had a 'familiar spreit'.  Hearsay evidence given in by a woman who 

In [83]:
case['MeetingNotes'].unique()

array([nan,
       "See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  See also Napier (c/egd/103) for more details about the North Berwick meeting. Meeting said to have taken place at the kirk at North Berwick, and a trump was played as a musical instrument.",
       "Claimed by others that she was part of the planning of the conspiracy to sink Queen Anne'sI ships on the way from Denmark.  Said to have been at meetings at Beigis Tod's house and the salt pans. See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  She was supposedly part of the baptism of a cat.",
       "See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  See also Napier (c/egd/103) for more details about the North Berwick meeting. Claimed that she attended meeting at the kirk in North Berwick and that a trump was played.",
       "Prior to the North Berwick description there is a detailed account of who was supposed to do what in preparati

In [85]:
case['FolkNotes'].unique()

array([nan,
       "In other accused's confession it was said that they had used corpse and corpse powder, and had met at halloween.",
       'She was included in the ritual to tie joints to the feet of a cat and then cast it into the sea to raise storms.',
       'Claimed by others that she was involved in using a corpse and powder from a corpse and attended meetings at Halloween.',
       'This case has some interesting details about the herbs and other ingredients that she used in salves. These included rippell grasse (ripple grass or robwort plantain), waffron leaf(?), plantain feather(?), finglie (?finkle or fennel), and hyssop(?). She denied she had used foxtree leaves (foxglove). She also claimed that she had her skill from the possession of a book, that had belonged to her father and grandfather. She said it was a 1000 years old but as she could not read it her son, Adam Bell, would read out bits to her. The book was taken from her during the ministry of William Couper who was 

In [87]:
case['CaseNotes'].unique()

array(['certain persons from Haymouth.', nan,
       'A very complicated case that moved back and forth from Edinburgh to Dysart during the course ofher two trials.  Definite factional politics at play amongst the bailies and ministers of Dysart. There is a source listed by MacDonald that we have not checked, which records a date of 30/8/1643.',
       'In 1626 she promised the presbytery she would refrain from using her cures, especially in the burgh of Perth. The 1628 accusations were in part a result of her infringement of this promise.',
       'Listed with 7 others accused of witchcraft.',
       'listed with 7 others.', 'Listed with 7 others.',
       'Listed with 7 others including her mother.',
       'Listed with 7 others including her daughter.',
       'Accused with 2 others but very few details.',
       'Accused with 2 others.', 'Brought into the North Berwick stuff.',
       'listed with 7 other accused.', 'Accused with 7 others.',
       "A funny note:  Margaret claimed 

In [91]:
trial = all_dfs['Trial']

In [95]:
trial['TrialNotes'].unique()

array([nan, 'No trial details.', 'No details', 'no trial details.',
       'No details.',
       'Did not appear for trial as she had escaped. Sentence of excommunication passed.',
       'No details of trial.',
       'There is a note in the presbyteryon 24/7/1550  that witches were to be tried on the following Monday in Renfrew but no details of outcome.',
       '24/7/1650, note in presbytery records that the trial was to be held the following Monday in Renfrew but no details of outcome.',
       'Note in presbytery records on 24/7/1650 that a trial was to be held the following Monday in Renfrew, but no record of verdict.',
       'Note in presbytery records on 24/7/1650 that a trial was to be held on the following Monday in Renfrew but no details about outcome.',
       'Note in presbytery records on 17/3/1697 that ministers and commissioners to meet tomorrow in Renfrew for trial (?).  Trial started on 13/4/1697 but was continued until 12/5/1697 it lasted until the 19th.',
       '

In [97]:
accused['Notes'].unique()

array([nan,
       'Has a daughter accused of witchcraft who was most likely not younger than 25, 25+25=50.',
       'The presbytery entries refer to an earlier incident in 1611 when Wright was reprimanded and told to stop using cures. Age calculated from career, 25 + 17 = 42 although she was probably older as her son was married. The bail she was given was for \xc2\xa31000 so she appears to have been middling economic status.',
       "confessed to being a witch for 18-19 years, 18+25=43. Can't find Nidrie parish in Linlithgow presbytery.",
       'few details about accused.',
       "He was the Servant to Earl of Bothwell.  There was a proclamation against him.  He is described as a 'servitor' to Francis, Earl of Bothwell, and he has his own servant.",
       'Few details.', 'Few details about accused.',
       'Age estimated from mother.',
       'No details about location of accused. In presbytery she is recorded as a vagabond but in RPC there is a blank for the place name.',
     

In [102]:
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import colorsys
import random

In [104]:
devil_notes = pd.DataFrame(case['DevilNotes'])

In [115]:
devil_notes = devil_notes.dropna()

In [124]:
devil_notes.values

array([["Said by others to have met the devil as a 'mekle black man'"],
       ['Conversing with the Devil'],
       ["Others confessed that she met the devil as a 'mekle black man'."],
       ['Baptised a cat.'],
       ['No details of pact.'],
       ['No pact details'],
       ['no details about pact recorded.'],
       ['paction confessed but no details.'],
       ['Pact confessed but no details.'],
       ['Presbytery notes confessed paction with the devil.'],
       ['Presbytery notes confessed paction with the devil.'],
       ['Few details but presbytery notes she confessed paction with the devil.'],
       ['Her servant saw a vision of a man in a white shirt with a tail.'],
       [ 'Confessed that he had seen the fairies in different forms but did not describe them.'],
       [ 'Dalkeith presbytery note the accused confessed to paction, witchcraft and charming.'],
       ['No details of pact.'],
       ['No details of pact.'],
       ['Devil gave her the power to heal.'],
   

In [108]:
devil = all_dfs['DevilAppearance']

In [114]:
devil.Devil_Text

0                                   fearful shape
1                               in green clothing
2                                large, black man
3                         in green with black hat
4                              young man in green
5      little man, in blue, blue hat, black staff
6                                  grim black man
7                                  black clothing
8                           green clothes/kirtall
9                      black man in black clothes
10                         green clothes/grey hat
11                                            NaN
12                                            NaN
13                               in brown clothes
14                                  black clothes
15                                            dog
16                                            NaN
17                               in black clothes
18                             black man in green
19                                      gentleman


In [111]:
elf = all_dfs["Elf_FairyElements"]

In [112]:
elf

,ElfFairyRef,ElfFairySystemId,ElfFairyID,CaseRef,ElfFairy_Type,Createdby,Createdate,Lastupdatedby,Lastupdatedon
0,EFT/JO/1,JO,1,C/JO/2632,Changeling,jhm,05/17/01 12:06:22,jhm,11/13/02 15:52:43
1,EFT/JO/100,JO,100,C/EGD/2312,Sunrise,jhm,06/19/02 13:52:05,jhm,06/19/02 13:52:08
2,EFT/JO/101,JO,101,C/JO/3044,Elfshot,jhm,06/19/02 14:16:21,jhm,06/19/02 14:16:22
3,EFT/JO/103,JO,103,C/JO/3059,Group of fairies,jhm,06/27/02 14:30:09,jhm,06/27/02 14:30:13
4,EFT/JO/104,JO,104,C/JO/3059,Changeling,jhm,06/27/02 14:30:15,jhm,06/27/02 14:30:18
5,EFT/JO/105,JO,105,C/JO/3068,Green,jhm,07/02/02 09:23:20,jhm,07/02/02 09:23:21
6,EFT/JO/106,JO,106,C/EGD/1045,Group of fairies,jhm,07/03/02 14:20:43,jhm,07/03/02 14:20:47
7,EFT/JO/107,JO,107,C/EGD/1045,Changeling,jhm,07/03/02 14:20:48,jhm,07/03/02 14:20:49
8,EFT/JO/108,JO,108,C/EGD/1045,Iron,jhm,07/03/02 14:20:53,jhm,07/03/02 14:20:57
9,EFT/JO/109,JO,109,C/EGD/2262,Thorn Tree,jhm,07/03/02 14:58:22,jhm,07/03/02 14:58:28
